In [198]:
from datetime import datetime
import pandas as pd
import numpy as np
import tensorflow as tf

STAGE 1 MODEL DATA ENGINEERING

In [199]:



hist = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\blantyre_2018-2024.csv", parse_dates=['time'])

# Ensure sorted by area and time for rolling computations
hist = hist.sort_values(['area','time']).reset_index(drop=True)

# -----------------------------
# 1. Rolling precipitation
# -----------------------------
hist['prcp_1d'] = hist.groupby('area')['prcp'].shift(1)  # yesterday's prcp
hist['prcp_3d'] = hist.groupby('area')['prcp'].rolling(3, min_periods=1).mean().reset_index(0,drop=True)
hist['prcp_7d'] = hist.groupby('area')['prcp'].rolling(7, min_periods=1).mean().reset_index(0,drop=True)
hist['prcp_30d'] = hist.groupby('area')['prcp'].rolling(30, min_periods=1).mean().reset_index(0,drop=True)

# Precipitation intensity ratio
hist['prcp_ratio'] = hist['prcp_1d'] / hist['prcp_7d']

# Antecedent Wetness Index (AWI)
hist['AWI'] = 0.6*hist['prcp_1d'] + 0.3*hist['prcp_3d'] + 0.1*hist['prcp_7d']


if 'pres' in hist.columns:
    hist['pres_24h_drop'] = hist.groupby('area')['pres'].shift(1) - hist['pres']




nino_train = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\nino2016_2023.csv", parse_dates=['date'])
nino_test  = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\nino2025.csv", parse_dates=['date'])


hist = hist.merge(nino_train.rename(columns={'date':'time'}), on='time', how='left')


stage1_features = ['time','area','prcp_1d','prcp_3d','prcp_7d','prcp_30d','prcp_ratio','AWI',
                   'pres','pres_24h_drop','wspd','nino34_sst_anomaly']


STAGE 2 DATA ENG

In [200]:
# Load topographic / spatial data
topo = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\Blantyre_data.csv")  # columns: area, elevation, slope, soil_moisture_index, dist_mz_border_km, dist_major_river_km



topo['elevation_x_slope'] = topo['elevation_m'] * topo['avg_slope_deg']


stage2_features = ['elevation','slope','elevation_x_slope',
                   'dist_mz_border_km','dist_major_river_km','month','lsi','ofi']


In [201]:
topo

,area,latitude,longitude,elevation_m,lsi,ofi,avg_slope_deg,distance_km_from_Mulanje,elevation_x_slope
0,Blantyre_CBD,-15.78610,35.00580,1030,0.38,0.52,6,59.96,6180
1,Chilobwe,-15.79250,35.00010,1015,0.55,0.78,20,60.20,20300
2,Ndirande,-15.72790,34.98630,1180,0.61,0.66,30,64.90,35400
3,Machinjiri,-15.77650,34.98720,1100,0.58,0.63,20,62.22,22000
4,Bangwe,-15.80410,34.98600,990,0.42,0.74,6,61.03,5940
5,Zingwangwa,-15.78950,34.97540,985,0.40,0.69,12,62.73,11820
6,Nancholi,-15.78840,35.00290,1008,0.46,0.62,12,60.13,12096
7,Chilomoni,-15.79880,35.00220,1002,0.44,0.67,12,59.69,12024
8,Mbayani,-15.79100,35.00750,1010,0.47,0.64,12,59.56,12120
9,Chirimba,-15.79120,35.00150,1005,0.45,0.60,6,60.12,6030


In [202]:


# Load forecast and parse Date
forecast = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\forecast_data.csv")
forecast['time'] = pd.to_datetime(forecast['Date'])  # create 'time' column
forecast = forecast.rename(columns={'Forecast':'prcp'})  # precipitation column

# Drop the original Date column if you want
forecast = forecast.drop(columns=['Date'])


In [203]:
nino_test = pd.read_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\nino2025.csv")
nino_test['time'] = pd.to_datetime(nino_test['date'])
nino_test = nino_test[['time','nino34_sst_anomaly']]  # keep only relevant columns


In [204]:
forecast = forecast.merge(nino_test, on='time', how='left')


In [205]:
forecast['area'] = 'Blantyre_CBD'

forecast = forecast.sort_values(['area','time']).reset_index(drop=True)

forecast['prcp_1d'] = forecast.groupby('area')['prcp'].shift(1)
forecast['prcp_3d'] = forecast.groupby('area')['prcp'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)
forecast['prcp_7d'] = forecast.groupby('area')['prcp'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)
forecast['prcp_30d'] = forecast.groupby('area')['prcp'].rolling(30, min_periods=1).mean().reset_index(0, drop=True)

forecast['prcp_ratio'] = forecast['prcp_1d'] / forecast['prcp_7d']
forecast['AWI'] = 0.6*forecast['prcp_1d'] + 0.3*forecast['prcp_3d'] + 0.1*forecast['prcp_7d']

# -----------------------------
# 5. Save ready-to-predict dataset
# -----------------------------
forecast.to_csv(r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\forecast_ready_2025_2026.csv",
                index=False)

print("Forecast dataset for 2025–2026 prepared with Stage 1 + Stage 2 features.")

Forecast dataset for 2025–2026 prepared with Stage 1 + Stage 2 features.


In [206]:
who

MinMaxScaler	 X1	 col	 datetime	 df	 df_clean	 drop_duplicate_columns	 drop_duplicate_columns_safe	 expit	 
forecast	 full	 full_data	 full_hist	 g	 hazard_features	 hist	 nino_test	 nino_train	 
np	 numeric_cols	 output	 p	 p_clean	 pd	 river	 scaler_h	 scaler_s	 
stage1_features	 stage2_features	 susceptibility_features	 tf	 topo	 zscore	 


2 staged model architecture

In [207]:
hist.head(5)

,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,prcp_1d,prcp_3d,prcp_7d,prcp_30d,prcp_ratio,AWI,pres_24h_drop,nino34_sst_anomaly
0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,0.0,4.450000,4.450000,4.450000,0.000000,1.780000,NaN,NaN
2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,8.9,7.533333,7.533333,7.533333,1.181416,8.353333,NaN,NaN
3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,13.7,7.533333,5.650000,5.650000,2.424779,11.045000,NaN,-0.73
4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,0.0,4.566667,4.520000,4.520000,0.000000,1.822000,NaN,NaN


In [208]:
print(topo.columns.tolist())

['area', 'latitude', 'longitude', 'elevation_m', 'lsi', 'ofi', 'avg_slope_deg', 'distance_km_from_Mulanje', 'elevation_x_slope']


In [209]:

from sklearn.preprocessing import MinMaxScaler

# --------------------------------------------------
# 1. Load prepared forecast dataset
# --------------------------------------------------
df = pd.read_csv(
    r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\full_data.csv",
    parse_dates=['time']
)
df.dropna()
# --------------------------------------------------
# 2. Stage 1: Hazard probability
# --------------------------------------------------
hazard_features = [
    'prcp_1d', 'prcp_3d', 'prcp_7d', 'prcp_30d',
    'prcp_ratio', 'AWI', 'nino34_sst_anomaly'
]

scaler_h = MinMaxScaler()
df[hazard_features] = scaler_h.fit_transform(df[hazard_features])

# Weighted hazard score (physically motivated)
df['hazard_prob'] = (
    0.30 * df['AWI'] +
    0.25 * df['prcp_3d'] +
    0.15 * df['prcp_7d'] +
    0.10 * df['prcp_ratio'] +
    0.10 * df['prcp_30d'] +
    0.10 * df['nino34_sst_anomaly']
)

df['hazard_prob'] = df['hazard_prob'].clip(0, 1)

# --------------------------------------------------
# 3. Stage 2: Physical susceptibility
#


C:\Users\USER\AppData\Local\Temp\ipykernel_14452\1835438507.py:6: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [210]:
df.head()

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,latitude,longitude,elevation_m,lsi,ofi,avg_slope_deg,distance_km_from_Mulanje,dist_to_water_km,elevation_x_slope,hazard_prob
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,-15.809,34.971,980,0.4,0.69,12,62.28,0.4,11760,NaN
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,-15.809,34.971,980,0.4,0.69,12,62.28,0.4,11760,NaN
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,-15.809,34.971,980,0.4,0.69,12,62.28,0.4,11760,NaN
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,-15.809,34.971,980,0.4,0.69,12,62.28,0.4,11760,0.087364
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,-15.809,34.971,980,0.4,0.69,12,62.28,0.4,11760,NaN


In [211]:
import numpy as np

# -------------------------------
# 1. Define columns to clean
# -------------------------------
numeric_cols = [
    'lsi', 'ofi',
    'dist_to_water_km',
    'elevation_x_slope',
    'hazard_prob'
]

# -------------------------------
# 2. Strip text + coerce to float
# -------------------------------
for col in numeric_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(',', '.', regex=False)
        .str.extract(r'([-+]?\d*\.?\d+)')[0]
        .astype(float)
    )

# -------------------------------
# 3. Handle missing / invalid values
# -------------------------------
df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# -------------------------------
# 4. Enforce physical bounds
# -------------------------------
df['lsi'] = df['lsi'].clip(0, 1)
df['ofi'] = df['ofi'].clip(0, 1)
df['hazard_prob'] = df['hazard_prob'].clip(0, 1)

# distances & topo must be non-negative
df['dist_to_water_km'] = df['dist_to_water_km'].clip(lower=0)
df['elevation_x_slope'] = df['elevation_x_slope'].clip(lower=0)

# -------------------------------
# 5. Final sanity check
# -------------------------------
assert df[numeric_cols].isna().sum().sum() == 0


In [212]:

from sklearn.preprocessing import MinMaxScaler

susceptibility_features = [
    'lsi', 'ofi',
    'dist_to_water_km', 'elevation_x_slope'
]

scaler_s = MinMaxScaler()
df[susceptibility_features] = scaler_s.fit_transform(df[susceptibility_features])

# Invert protective variables
df['inv_elev_slope'] = 1 - df['elevation_x_slope']
df['inv_river_dist'] = 1 - df['dist_to_water_km']

df['susceptibility'] = (
    0.30 * df['lsi'] +
    0.25 * df['ofi'] +
    0.15 * df['inv_river_dist'] +
    0.10 * df['inv_elev_slope']
)

df['susceptibility'] = df['susceptibility'].clip(0, 1)

# --------------------------------------------------
# 4. Final flood risk probability
# --------------------------------------------------
df['flood_risk_probability'] = (
    df['hazard_prob'] * df['susceptibility']
).clip(0, 1)

# --------------------------------------------------
# 5. Area-wise daily output
# --------------------------------------------------
output = df[['time', 'area', 'flood_risk_probability']]

output.to_csv(
    r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\flood_risk_probabilities_2025_2026.csv",
    index=False
)

print("Flood risk probabilities computed for all areas.")

Flood risk probabilities computed for all areas.


In [213]:
df

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,ofi,avg_slope_deg,distance_km_from_Mulanje,dist_to_water_km,elevation_x_slope,hazard_prob,inv_elev_slope,inv_river_dist,susceptibility,flood_risk_probability
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261


In [214]:
g = df.groupby('area')

df['prcp_1d'] = g['prcp'].shift(1)
df['prcp_3d'] = g['prcp'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)
df['prcp_7d'] = g['prcp'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)
df['prcp_30d'] = g['prcp'].rolling(30, min_periods=1).mean().reset_index(0, drop=True)

df['AWI'] = (
    0.6 * df['prcp_1d'].fillna(0) +
    0.3 * df['prcp_3d'] +
    0.1 * df['prcp_7d']
)

df['prcp_ratio'] = (
    df['prcp_1d'] /
    (df['prcp_7d'] + 1e-6)
)


In [215]:
for col in ['pres','pres_24h_drop','wspd','tmin','tmax']:
    df[col] = (
        df
        .groupby('area')[col]
        .transform(lambda x: x.interpolate(limit=2))
    )


In [216]:
g = df.groupby('area')

df['prcp_7d'] = (
    g['prcp']
    .rolling(7, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)


In [217]:
df['prcp_1d'] = (
    df
    .groupby('area')['prcp']
    .shift(1)
    .fillna(0)
)


In [218]:
df['prcp'] = df['prcp'].fillna(0)

df.isna().mean().sort_values(ascending=False)



nino34_sst_anomaly          0.971819
prcp_ratio                  0.473270
AWI                         0.394530
prcp_3d                     0.394530
prcp_7d                     0.306672
prcp_30d                    0.134273
pres_24h_drop               0.024036
tmax                        0.023622
wspd                        0.023622
pres                        0.023622
prcp                        0.000000
tmin                        0.000000
time                        0.000000
area                        0.000000
tavg                        0.000000
Unnamed: 0                  0.000000
prcp_1d                     0.000000
Unnamed: 0.1                0.000000
latitude                    0.000000
longitude                   0.000000
elevation_m                 0.000000
lsi                         0.000000
ofi                         0.000000
avg_slope_deg               0.000000
distance_km_from_Mulanje    0.000000
dist_to_water_km            0.000000
elevation_x_slope           0.000000
h

In [219]:
full_data = df.sort_values(['area','time']).reset_index(drop=True)


In [220]:
full_data

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,ofi,avg_slope_deg,distance_km_from_Mulanje,dist_to_water_km,elevation_x_slope,hazard_prob,inv_elev_slope,inv_river_dist,susceptibility,flood_risk_probability
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261


In [221]:
df

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,ofi,avg_slope_deg,distance_km_from_Mulanje,dist_to_water_km,elevation_x_slope,hazard_prob,inv_elev_slope,inv_river_dist,susceptibility,flood_risk_probability
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,0.71875,12,62.28,0.2,0.105703,0.087364,0.894297,0.8,0.426617,0.037271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.71875,12,62.73,0.2,0.106793,0.087364,0.893207,0.8,0.426508,0.037261


In [222]:
def zscore(s):
    return (s - s.mean()) / (s.std() + 1e-6)

full_data['R_int'] = zscore(full_data['prcp_ratio'])
full_data['R_mem'] = zscore(full_data['AWI'] + full_data['prcp_7d'] + full_data['prcp_30d'])
full_data['Topo']  = zscore(full_data['elevation_x_slope'])
full_data['Drain'] = zscore(1 / (full_data['dist_to_water_km'] + 0.1))
full_data['Cycl'] = zscore(full_data['wspd'] + full_data['pres_24h_drop'])


In [223]:
full_data['hazard_raw'] = (
    0.35 * full_data['R_int'] +     # short-term intensity
    0.30 * full_data['R_mem'] +     # antecedent wetness memory
    0.20 * full_data['Topo'] +      # runoff acceleration
    0.10 * full_data['Drain'] +     # channel proximity
    0.05 * full_data['Cycl']        # synoptic forcing
)


In [224]:
full_data

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,inv_elev_slope,inv_river_dist,susceptibility,flood_risk_probability,R_int,R_mem,Topo,Drain,Cycl,hazard_raw
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,0.894297,0.8,0.426617,0.037271,NaN,-0.564353,-0.276753,0.178321,NaN,NaN
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,0.894297,0.8,0.426617,0.037271,-0.531130,-0.082957,-0.276753,0.178321,NaN,NaN
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,0.894297,0.8,0.426617,0.037271,0.286311,0.491292,-0.276753,0.178321,NaN,NaN
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,0.894297,0.8,0.426617,0.037271,1.146613,0.442837,-0.276753,0.178321,NaN,NaN
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,0.894297,0.8,0.426617,0.037271,-0.531130,-0.074754,-0.276753,0.178321,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.893207,0.8,0.426508,0.037261,3.926736,0.265989,-0.272021,0.178321,0.589630,1.447064
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.893207,0.8,0.426508,0.037261,-0.531130,-0.237300,-0.272021,0.178321,0.851577,-0.251079
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.893207,0.8,0.426508,0.037261,-0.531130,-0.373013,-0.272021,0.178321,-0.046526,-0.336698
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.893207,0.8,0.426508,0.037261,-0.531130,-0.343183,-0.272021,0.178321,-0.607840,-0.355814


In [225]:
from scipy.special import expit

full_data['flood_risk_prob'] = expit(full_data['hazard_raw'])


In [226]:
full_data["flood_risk_prob"].value_counts()

flood_risk_prob
0.447089    2
0.487228    2
0.589476    2
0.375052    2
0.498897    2
           ..
0.809546    1
0.437558    1
0.416612    1
0.411973    1
0.631493    1
Name: count, Length: 33835, dtype: int64

In [227]:
full_data.loc[full_data['prcp_1d'] < 5, 'flood_risk_prob'] *= 0.2
full_data.loc[full_data['prcp_7d'] < 10, 'flood_risk_prob'] *= 0.5


In [228]:
full_data

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,inv_river_dist,susceptibility,flood_risk_probability,R_int,R_mem,Topo,Drain,Cycl,hazard_raw,flood_risk_prob
0,0,2018-01-04,Angelogothere,0,23.8,20.1,NaN,0.0,NaN,NaN,...,0.8,0.426617,0.037271,NaN,-0.564353,-0.276753,0.178321,NaN,NaN,NaN
1,1,2018-01-18,Angelogothere,1,24.4,19.6,NaN,8.9,NaN,NaN,...,0.8,0.426617,0.037271,-0.531130,-0.082957,-0.276753,0.178321,NaN,NaN,NaN
2,2,2018-01-31,Angelogothere,2,26.2,21.6,NaN,13.7,NaN,NaN,...,0.8,0.426617,0.037271,0.286311,0.491292,-0.276753,0.178321,NaN,NaN,NaN
3,3,2018-02-01,Angelogothere,3,26.6,20.3,NaN,0.0,NaN,NaN,...,0.8,0.426617,0.037271,1.146613,0.442837,-0.276753,0.178321,NaN,NaN,NaN
4,4,2018-02-05,Angelogothere,4,25.6,20.8,NaN,0.0,NaN,NaN,...,0.8,0.426617,0.037271,-0.531130,-0.074754,-0.276753,0.178321,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.8,0.426508,0.037261,3.926736,0.265989,-0.272021,0.178321,0.589630,1.447064,0.404773
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.8,0.426508,0.037261,-0.531130,-0.237300,-0.272021,0.178321,0.851577,-0.251079,0.043756
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.8,0.426508,0.037261,-0.531130,-0.373013,-0.272021,0.178321,-0.046526,-0.336698,0.041661
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.8,0.426508,0.037261,-0.531130,-0.343183,-0.272021,0.178321,-0.607840,-0.355814,0.041197


In [230]:
full_data.columns


Index(['Unnamed: 0.1', 'time', 'area', 'Unnamed: 0', 'tavg', 'tmin', 'tmax',
       'prcp', 'wspd', 'pres', 'prcp_1d', 'prcp_3d', 'prcp_7d', 'prcp_30d',
       'prcp_ratio', 'AWI', 'pres_24h_drop', 'nino34_sst_anomaly', 'latitude',
       'longitude', 'elevation_m', 'lsi', 'ofi', 'avg_slope_deg',
       'distance_km_from_Mulanje', 'dist_to_water_km', 'elevation_x_slope',
       'hazard_prob', 'inv_elev_slope', 'inv_river_dist', 'susceptibility',
       'flood_risk_probability', 'R_int', 'R_mem', 'Topo', 'Drain', 'Cycl',
       'hazard_raw', 'flood_risk_prob'],
      dtype='object')

In [232]:
full_data.isna().sum().sort_values(ascending=False)


nino34_sst_anomaly          65660
flood_risk_prob             33572
hazard_raw                  33572
prcp_ratio                  31976
R_int                       31976
R_mem                       26656
AWI                         26656
prcp_3d                     26656
prcp_7d                     20720
prcp_30d                     9072
pres_24h_drop                1624
Cycl                         1624
wspd                         1596
tmax                         1596
pres                         1596
tmin                            0
tavg                            0
Unnamed: 0                      0
area                            0
Unnamed: 0.1                    0
time                            0
prcp                            0
prcp_1d                         0
ofi                             0
lsi                             0
elevation_m                     0
longitude                       0
latitude                        0
avg_slope_deg                   0
dist_to_water_

In [233]:
full_data = full_data.fillna(df.median(numeric_only=True))


In [234]:
full_data

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,inv_river_dist,susceptibility,flood_risk_probability,R_int,R_mem,Topo,Drain,Cycl,hazard_raw,flood_risk_prob
0,0,2018-01-04,Angelogothere,0,23.8,20.1,28.2,0.0,10.6,1016.4,...,0.8,0.426617,0.037271,NaN,-0.564353,-0.276753,0.178321,NaN,NaN,NaN
1,1,2018-01-18,Angelogothere,1,24.4,19.6,28.2,8.9,10.6,1016.4,...,0.8,0.426617,0.037271,-0.531130,-0.082957,-0.276753,0.178321,NaN,NaN,NaN
2,2,2018-01-31,Angelogothere,2,26.2,21.6,28.2,13.7,10.6,1016.4,...,0.8,0.426617,0.037271,0.286311,0.491292,-0.276753,0.178321,NaN,NaN,NaN
3,3,2018-02-01,Angelogothere,3,26.6,20.3,28.2,0.0,10.6,1016.4,...,0.8,0.426617,0.037271,1.146613,0.442837,-0.276753,0.178321,NaN,NaN,NaN
4,4,2018-02-05,Angelogothere,4,25.6,20.8,28.2,0.0,10.6,1016.4,...,0.8,0.426617,0.037271,-0.531130,-0.074754,-0.276753,0.178321,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.8,0.426508,0.037261,3.926736,0.265989,-0.272021,0.178321,0.589630,1.447064,0.404773
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.8,0.426508,0.037261,-0.531130,-0.237300,-0.272021,0.178321,0.851577,-0.251079,0.043756
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.8,0.426508,0.037261,-0.531130,-0.373013,-0.272021,0.178321,-0.046526,-0.336698,0.041661
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.8,0.426508,0.037261,-0.531130,-0.343183,-0.272021,0.178321,-0.607840,-0.355814,0.041197


In [240]:
from scipy.special import expit

full_data['trigger_raw'] = (
    0.40 * full_data['R_int'] +      # rainfall intensity
    0.35 * full_data['R_mem'] +      # antecedent wetness
    0.15 * full_data['Cycl'] +       # pressure + wind
    0.10 * full_data['nino34_sst_anomaly'].fillna(0)
)

full_data['trigger_prob'] = expit(full_data['trigger_raw'])


In [ ]:
full_data = full_data.dropnafull_data = full_data.fillna(0)


C:\Users\USER\AppData\Local\Temp\ipykernel_14452\2494995505.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  full_data = full_data.dropnafull_data = full_data.fillna(0)


In [241]:
full_data

,Unnamed: 0.1,time,area,Unnamed: 0,tavg,tmin,tmax,prcp,wspd,pres,...,flood_risk_probability,R_int,R_mem,Topo,Drain,Cycl,hazard_raw,flood_risk_prob,trigger_raw,trigger_prob
0,0,2018-01-04,Angelogothere,0,23.8,20.1,28.2,0.0,10.6,1016.4,...,0.037271,0.000000,-0.564353,-0.276753,0.178321,0.000000,0.000000,0.000000,-0.165492,0.458721
1,1,2018-01-18,Angelogothere,1,24.4,19.6,28.2,8.9,10.6,1016.4,...,0.037271,-0.531130,-0.082957,-0.276753,0.178321,0.000000,0.000000,0.000000,-0.209456,0.447827
2,2,2018-01-31,Angelogothere,2,26.2,21.6,28.2,13.7,10.6,1016.4,...,0.037271,0.286311,0.491292,-0.276753,0.178321,0.000000,0.000000,0.000000,0.318508,0.578961
3,3,2018-02-01,Angelogothere,3,26.6,20.3,28.2,0.0,10.6,1016.4,...,0.037271,1.146613,0.442837,-0.276753,0.178321,0.000000,0.000000,0.000000,0.628013,0.652039
4,4,2018-02-05,Angelogothere,4,25.6,20.8,28.2,0.0,10.6,1016.4,...,0.037271,-0.531130,-0.074754,-0.276753,0.178321,0.000000,0.000000,0.000000,-0.206584,0.448537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67559,67559,2024-12-27,Zingwangwa,2408,28.1,23.2,32.0,0.0,11.8,1015.3,...,0.037261,3.926736,0.265989,-0.272021,0.178321,0.589630,1.447064,0.404773,1.784266,0.856223
67560,67560,2024-12-28,Zingwangwa,2409,29.1,23.3,33.9,0.0,11.3,1013.3,...,0.037261,-0.531130,-0.237300,-0.272021,0.178321,0.851577,-0.251079,0.043756,-0.135739,0.466117
67561,67561,2024-12-29,Zingwangwa,2410,26.6,21.7,33.4,0.0,11.0,1013.4,...,0.037261,-0.531130,-0.373013,-0.272021,0.178321,-0.046526,-0.336698,0.041661,-0.317954,0.421174
67562,67562,2024-12-30,Zingwangwa,2411,26.6,24.3,31.0,2.3,9.3,1013.3,...,0.037261,-0.531130,-0.343183,-0.272021,0.178321,-0.607840,-0.355814,0.041197,-0.391711,0.403306


In [248]:
full_data['susceptibility_raw'] = (
    0.35 * full_data['Topo'] +
    0.25 * full_data['Drain'] +
    0.20 * full_data['lsi'] +
    0.20 * full_data['ofi']
)

full_data['susceptibility_prob'] = expit(full_data['susceptibility_raw'])


In [249]:
full_data['flood_risk_prob'] = (
    full_data['trigger_prob'] *
    full_data['susceptibility_prob']
)


In [250]:
full_data['flood_risk_prob']

0        0.242702
1        0.236938
2        0.306319
3        0.344983
4        0.237314
           ...   
67559    0.453367
67560    0.246807
67561    0.223010
67562    0.213549
67563    0.248299
Name: flood_risk_prob, Length: 67564, dtype: float64

Stage 1

In [253]:
stage1_features = [
    'prcp_1d',
    'prcp_3d',
    'prcp_7d',
    'prcp_30d',
    'prcp_ratio',
    'AWI',
    'pres_24h_drop',
    'wspd',
    'nino34_sst_anomaly'
]

X = full_data[stage1_features].copy()
y = full_data['hazard_raw']          # or expit(hazard_raw)


In [254]:
from xgboost import XGBRegressor

stage1_model = XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

stage1_model.fit(X, y)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [255]:
from scipy.special import expit

full_data['trigger_prob_ml'] = expit(stage1_model.predict(X))


In [256]:
full_data.loc[full_data['prcp_1d'] < 2, 'trigger_prob_ml'] *= 0.1
full_data.loc[full_data['prcp_7d'] < 10, 'trigger_prob_ml'] *= 0.4


In [257]:
full_data[['prcp_1d','prcp_7d','trigger_prob_ml']].describe()


,prcp_1d,prcp_7d,trigger_prob_ml
count,67564.000000,67564.000000,67564.000000
mean,2.123717,3.213313,0.062977
std,10.583758,7.900742,0.133522
min,0.000000,0.000000,0.015922
25%,0.000000,0.166667,0.018103
50%,0.000000,0.771429,0.019949
75%,0.000000,2.957143,0.020074
max,300.000000,160.000000,0.996134


In [262]:
freddy_probs = (
    full_data[
        (full_data['time'] >= '2023-02-01') &
        (full_data['time'] <= '2023-03-15')
    ]
    .groupby('time')['trigger_prob_ml']
    .mean()
)

# Convert to JSON
freddy_json = freddy_probs.to_json(
    orient='index',   # { "2023-02-01": 0.34, ... }
    date_format='iso'
)

print(freddy_json)


{"2023-02-01T00:00:00.000":0.2843866944,"2023-02-02T00:00:00.000":0.2418828309,"2023-02-03T00:00:00.000":0.0191988163,"2023-02-04T00:00:00.000":0.0186628085,"2023-02-05T00:00:00.000":0.0188149381,"2023-02-06T00:00:00.000":0.0187081043,"2023-02-07T00:00:00.000":0.2469737083,"2023-02-08T00:00:00.000":0.2325325757,"2023-02-09T00:00:00.000":0.0186149813,"2023-02-10T00:00:00.000":0.194540903,"2023-02-11T00:00:00.000":0.2160308361,"2023-02-12T00:00:00.000":0.2458041161,"2023-02-13T00:00:00.000":0.0188883506,"2023-02-14T00:00:00.000":0.0195320938,"2023-02-15T00:00:00.000":0.2243149281,"2023-02-16T00:00:00.000":0.0199669432,"2023-02-17T00:00:00.000":0.2828660607,"2023-02-18T00:00:00.000":0.2040052712,"2023-02-19T00:00:00.000":0.0191409793,"2023-02-20T00:00:00.000":0.28804636,"2023-02-21T00:00:00.000":0.0189414527,"2023-02-22T00:00:00.000":0.0185448229,"2023-02-23T00:00:00.000":0.0189737976,"2023-02-24T00:00:00.000":0.2381073236,"2023-02-25T00:00:00.000":0.0188663043,"2023-02-26T00:00:00.000":0

In [ ]:

from sklearn.preprocessing import MinMaxScaler

susceptibility_features = [
    'lsi', 'ofi',
    'dist_to_water_km', 'elevation_x_slope'
]

scaler_s = MinMaxScaler()
df[susceptibility_features] = scaler_s.fit_transform(df[susceptibility_features])

# Invert protective variables
df['inv_elev_slope'] = 1 - df['elevation_x_slope']
df['inv_river_dist'] = 1 - df['dist_to_water_km']

df['susceptibility'] = (
    0.30 * df['lsi'] +
    0.25 * df['ofi'] +
    0.15 * df['inv_river_dist'] +
    0.10 * df['inv_elev_slope']
)

df['susceptibility'] = df['susceptibility'].clip(0, 1)

# --------------------------------------------------
# 4. Final flood risk probability
# --------------------------------------------------
df['flood_risk_probability'] = (
    df['hazard_prob'] * df['susceptibility']
).clip(0, 1)

# --------------------------------------------------
# 5. Area-wise daily output
# --------------------------------------------------
output = df[['time', 'area', 'flood_risk_probability']]

output.to_csv(
    r"D:\Backup folder\Code\GitHub\goesafe101\AI\data\flood_risk_probabilities_2025_2026.csv",
    index=False
)

print("Flood risk probabilities computed for all areas.")

Flood risk probabilities computed for all areas.
